In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import TensorBoard
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from time import time
import pickle

In [3]:
# Load dataset
df = pd.read_excel('data_num.xlsx')
df

,Time,Temperature,Wind,Visibility,Snow on Ground,Min Delay
0,0.695833,-17.72,26.88,14.4708,3,3
1,0.776389,-17.72,26.88,14.4708,3,4
2,0.271528,-17.72,26.88,14.4708,3,4
3,0.735417,-17.72,26.88,14.4708,3,7
4,0.598611,-17.72,26.88,14.4708,3,7
...,...,...,...,...,...,...
76264,0.229861,0.15,23.50,17.8250,1,20
76265,0.340278,0.15,23.50,17.8250,1,20
76266,0.553472,0.15,23.50,17.8250,1,20
76267,0.972222,0.15,23.50,17.8250,1,20


In [4]:
# Unfiltered dataset
df.describe()

,Time,Temperature,Wind,Visibility,Snow on Ground,Min Delay
count,76269.000000,76269.000000,76269.000000,76269.000000,76269.000000,76269.000000
mean,0.506089,8.333007,16.980718,21.727279,1.671203,13.106596
std,0.241679,11.619888,6.601680,3.789496,4.569954,30.666536
min,0.000000,-21.950000,5.540000,4.070800,0.000000,1.000000
25%,0.293056,-0.490000,11.790000,20.662500,0.000000,5.000000
50%,0.497917,8.850000,15.880000,23.766700,0.000000,6.000000
75%,0.695833,18.710000,21.290000,24.100000,0.000000,12.000000
max,0.999306,29.380000,44.580000,26.450000,38.000000,1400.000000


In [5]:
# Min delay has outliers, process dataset to remove outliers
col = ['Min Delay']

# Calculate quantiles and IQR
Q1 = df[col].quantile(0.25)
Q3 = df[col].quantile(0.75)
IQR = Q3 - Q1

# Filter outliers
df = df[~((df[col] < (Q1 - 1.5 * IQR)) |(df[col] > (Q3 + 1.5 * IQR))).any(axis=1)]

In [6]:
# Filtered dataset
df.describe()

,Time,Temperature,Wind,Visibility,Snow on Ground,Min Delay
count,67624.000000,67624.000000,67624.000000,67624.000000,67624.000000,67624.000000
mean,0.500825,8.209480,17.001693,21.732335,1.675603,7.395895
std,0.238940,11.654371,6.620984,3.784942,4.531596,4.417649
min,0.000000,-21.950000,5.540000,4.070800,0.000000,1.000000
25%,0.289583,-0.620000,11.790000,20.662500,0.000000,4.000000
50%,0.486111,8.790000,15.880000,23.766700,0.000000,6.000000
75%,0.687500,18.640000,21.420000,24.100000,0.000000,10.000000
max,0.999306,29.380000,44.580000,26.450000,38.000000,22.000000


In [7]:
# Split into 80/20 train/test sets
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False)

In [8]:
# Separate train and test datasets into input and output
train_input = train_df.copy()
train_output = train_input.pop('Min Delay')
test_input = test_df.copy()
test_output = test_input.pop('Min Delay')

In [9]:
# Scale input to 0-1 using MinMaxScaler
cols = train_input.columns
scaler = MinMaxScaler()

scaled_train_input = pd.DataFrame(scaler.fit_transform(train_input), columns=cols)
scaled_test_input = pd.DataFrame(scaler.transform(test_input), columns=cols)

In [10]:
# Print scaled train input dataset
scaled_train_input.describe()

,Time,Temperature,Wind,Visibility,Snow on Ground
count,54099.000000,54099.000000,54099.000000,54099.000000,54099.000000
mean,0.499525,0.594760,0.296612,0.794601,0.067241
std,0.237667,0.228897,0.174368,0.166340,0.180243
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.287700,0.422560,0.157007,0.752002,0.000000
50%,0.481584,0.606663,0.261328,0.886055,0.000000
75%,0.685198,0.797389,0.418335,0.894992,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [11]:
# Print scaled test input dataset
scaled_test_input.describe()

,Time,Temperature,Wind,Visibility,Snow on Ground
count,13525.000000,13525.000000,13525.000000,13525.000000,13525.000000
mean,0.507766,0.558763,0.323258,0.767567,0.095299
std,0.244676,0.217131,0.172999,0.178218,0.252031
min,0.000000,0.064485,0.024236,0.109664,0.000000
25%,0.300903,0.401909,0.206533,0.668590,0.000000
50%,0.504517,0.535359,0.293203,0.865201,0.000000
75%,0.701181,0.755504,0.418335,0.894992,0.000000
max,1.000000,0.969608,1.028451,0.894992,1.652174


In [12]:
# Convert to numpy arrays
x_train = scaled_train_input.to_numpy(dtype='float32')
y_train = train_output.to_numpy(dtype='float32')
x_test = scaled_test_input.to_numpy(dtype='float32')
y_test = test_output.to_numpy(dtype='float32')

In [13]:
# Define DNN model
def build_model(hp):  # random search passes this hyperparameter() object 
    model = keras.models.Sequential()
    
    # Varies num of neurons in the layer
    model.add(Dense(hp.Int('input_units', min_value=32, max_value=256, step=32)))
    model.add(Activation('relu'))

    for i in range(hp.Int('n_layers', 1, 4)):  # adding variation of num of layers.
        model.add(Dense(hp.Int(f'dense_{i}_units', min_value=32, max_value=256, step=32)))
        model.add(Activation('relu'))
    
    model.add(Dense(1))
    
    # Choose learning rate value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss="mean_squared_error",
                  metrics=["mae"])
    
    return model

In [14]:
LOG_DIR = f"logs/{int(time())}"
tensorboard = TensorBoard(log_dir=LOG_DIR)

# Hyperparameter tuning using keras_tuner
tuner = RandomSearch(
    build_model,
    objective='val_mae',
    max_trials=200,  # how many model variations to test?
    executions_per_trial=3,  # how many trials per variation? (same model could perform differently)
    directory=LOG_DIR)

tuner.search(x=x_train,
             y=y_train,
             verbose=2,
             epochs=10,
             batch_size=64,
             callbacks=[tensorboard],
             validation_data=(x_test, y_test))


Trial 200 Complete [00h 00m 36s]
val_mae: 3.1214128335316977

Best val_mae So Far: 3.058905919392904
Total elapsed time: 02h 03m 02s
INFO:tensorflow:Oracle triggered exit


In [17]:
# Print summary of hyperparameter search space
tuner.search_space_summary()

Search space summary
Default search space size: 7
input_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
n_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
dense_0_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
dense_1_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dense_2_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
dense_3_units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}


In [15]:
# Print hyperparameter values of best performing model
tuner.get_best_hyperparameters()[0].values

{'input_units': 32,
 'n_layers': 3,
 'dense_0_units': 64,
 'learning_rate': 0.01,
 'dense_1_units': 224,
 'dense_2_units': 96,
 'dense_3_units': 224}

In [18]:
# Summary of 10 best trials out of 200
tuner.results_summary()

Results summary
Results in logs/1658343178\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
input_units: 32
n_layers: 3
dense_0_units: 64
learning_rate: 0.01
dense_1_units: 224
dense_2_units: 96
dense_3_units: 224
Score: 3.058905919392904
Trial summary
Hyperparameters:
input_units: 64
n_layers: 1
dense_0_units: 96
learning_rate: 0.01
dense_1_units: 64
dense_2_units: 224
dense_3_units: 96
Score: 3.0589710076649985
Trial summary
Hyperparameters:
input_units: 160
n_layers: 2
dense_0_units: 128
learning_rate: 0.01
dense_1_units: 32
dense_2_units: 224
dense_3_units: 224
Score: 3.0600473880767822
Trial summary
Hyperparameters:
input_units: 64
n_layers: 3
dense_0_units: 32
learning_rate: 0.01
dense_1_units: 96
dense_2_units: 64
dense_3_units: 192
Score: 3.060138702392578
Trial summary
Hyperparameters:
input_units: 224
n_layers: 1
dense_0_units: 160
learning_rate: 0.01
dense_1_units: 96
dense_2_units: 96
dense_3_units: 128
Score: 3.0647040208180747
Trial summary
Hyperpara

In [ ]:
# Save tuner settings using pickle
with open(f"tuner_{int(time())}.pkl", "wb") as f:
    pickle.dump(tuner, f)